한국어 데이터에 대한 색인/검색

### 3.1 Pseudo train dataset 생성-> LLM으로 별도로 문서 1개당 질의 3개씩 추출

In [ ]:
from openai import OpenAI
import os
import json

from openai import OpenAI  # OpenAI API를 사용하기 위한 모듈 가져오기
import traceback  # 예외 발생 시 오류 추적을 위한 모듈 가져오기
from dotenv import load_dotenv
import os

# OPENAI_API_KEY.env 파일 생성

# OPENAI_API_KEY = "your API key here"

# 특정 .env 파일 경로를 지정하여 환경 변수 로드
load_dotenv(dotenv_path='/data/ephemeral/home/upstage-ai-advanced-ir2/OPENAI_API_KEY.env')

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()  # OpenAI 클라이언트 생성


In [1]:
# import json

# # 파일 경로
# input_path = '/data/ephemeral/home/upstage-ai-advanced-ir2/data/combined_crossencoder.jsonl'
# output_path = '/data/ephemeral/home/upstage-ai-advanced-ir2/data/modified_crossencoder.jsonl'

# # JSONL 파일 읽기
# def read_jsonl(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         return [json.loads(line) for line in file]

# # 파일 쓰기
# def write_jsonl(file_path, data):
#     with open(file_path, 'w', encoding='utf-8') as file:
#         for entry in data:
#             file.write(json.dumps(entry, ensure_ascii=False) + '\n')

# # 입력 파일 읽기
# combined_data = read_jsonl(input_path)

# # 새 파일의 형식으로 변환
# new_data = []
# current_content = None
# current_queries = []

# for entry in combined_data:
#     content = entry["content"]
#     query = entry["query"]
    
#     # 같은 content에 대해 queries를 배열로 묶음
#     if content == current_content:
#         current_queries.append(query)
#     else:
#         if current_content is not None:
#             # 이전 문서를 저장
#             new_data.append({"query": current_queries})
#         # 새 문서 시작
#         current_content = content
#         current_queries = [query]

# # 마지막 문서 저장
# if current_content is not None:
#     new_data.append({"query": current_queries})

# # 결과를 JSONL 파일로 저장
# write_jsonl(output_path, new_data)

# print(f'Modified file saved at {output_path}')


Modified file saved at /data/ephemeral/home/upstage-ai-advanced-ir2/data/modified_crossencoder.jsonl


In [1]:
# query 데이터를 json으로 저장해서 jsonl로 변경하는 코드, json파일 형식: [{'query': ["query1", "query2", "query3"]},...]
import json

# 파일 경로 설정
input_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/colbert/queries.json'
output_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/colbert/queries.jsonl'

# JSON 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# JSONL 파일로 변환하여 저장
with open(output_file, 'w', encoding='utf-8') as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"JSONL 파일로 변환 완료: {output_file}")


JSONL 파일로 변환 완료: /data/ephemeral/home/upstage-ai-advanced-ir2/colbert/queries.jsonl


In [ ]:
# collection 변수 (문서)
import json
with open("/data/ephemeral/home/upstage-ai-advanced-ir2/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
# docs 에서 줄바꿈 문자를 제거 (학습 할때 에러 방지)
for doc in docs:
    doc['content'] = doc['content'].replace("\t", "").replace("\n", "").replace("\r", "")

sample_ko_collection = [doc['content'] for doc in docs]
sample_ko_collection

In [3]:
# # \n, \t, \r을 제거하는 함수
# def clean_text(text):
#     # \n, \t, \r 제거
#     return text.replace('\n', '').replace('\t', '').replace('\r', '')



# # 리스트 내 모든 문장에 대해 \n, \t, \r 제거
# sample_ko_collection = [clean_text(sentence) for sentence in sample_collection]

# # 결과 출력
# print(sample_ko_collection[:5])

In [3]:
import random

# 파일 경로 설정 (queries.jsonl 파일)
query_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/data/modified_crossencoder.jsonl'

# queries.jsonl 파일에서 데이터를 읽어오기
query_data = []
with open(query_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        query_data.extend(data['query'])

# sample_ko_collection은 이미 존재하는 데이터로 가정
triples_data = []
q_idx = 0
max_c_idx = len(sample_ko_collection) - 1

for c_idx, sample in enumerate(sample_ko_collection):
    # 각 sample에 대해 query_data에서 쿼리 3개씩 사용
    for i in range(3):  # 3개의 쿼리만 사용
        if q_idx >= len(query_data):
            break  # 쿼리 데이터가 부족하면 중단
        q = query_data[q_idx]
        
        # mc_idx는 c_idx와 달라야 함
        mc_idx = random.randint(0, max_c_idx)
        while mc_idx == c_idx:
            mc_idx = random.randint(0, max_c_idx)

        # triples_data에 추가
        triples_data.append(f'{q_idx}, {c_idx}, {mc_idx}')
        q_idx += 1

# 결과 출력 (처음 5개)
print(triples_data[:10])
print(query_data[:5])


['0, 0, 1576', '1, 0, 2792', '2, 0, 41', '3, 1, 1715', '4, 1, 701', '5, 1, 1145', '6, 2, 3089', '7, 2, 905', '8, 2, 712', '9, 3, 522']
['에너지 균형이란 무엇인가요?', '건강한 사람이 에너지 균형을 유지해야 하는 이유는 무엇인가요?', '에너지 균형을 달성하기 위한 일반적인 기간은 얼마인가요?', '에너지 섭취와 에너지 소비를 조절하는 방법은 무엇인가요?', '영양가 있는 식품의 예시는 무엇인가요?']


In [4]:
import random
import json

# 파일 경로 설정 (queries.jsonl 파일)
query_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/data/modified_crossencoder.jsonl'

# queries.jsonl 파일에서 데이터를 읽어오기
query_data = []
with open(query_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        query_data.extend(data['query'])

# sample_ko_collection은 이미 존재하는 데이터로 가정
triples_data = []
q_idx = 0
max_c_idx = len(sample_ko_collection) - 1

# c_idx를 13개마다 증가시키기 위한 변수
c_idx = 0
query_count = 0  # 현재까지 처리한 쿼리 수

while q_idx < len(query_data):
    # 각 sample에 대해 query_data에서 쿼리 3개씩 사용
    for i in range(3):  # 3개의 쿼리만 사용
        if q_idx >= len(query_data):
            break  # 쿼리 데이터가 부족하면 중단
        q = query_data[q_idx]

        # mc_idx는 c_idx와 달라야 함
        mc_idx = random.randint(0, max_c_idx)
        while mc_idx == c_idx:
            mc_idx = random.randint(0, max_c_idx)

        # triples_data에 추가
        triples_data.append(f'{q_idx}, {c_idx}, {mc_idx}')
        q_idx += 1
        query_count += 1

        # 13개마다 c_idx 증가
        if query_count == 13:
            c_idx += 1
            query_count = 0  # 다시 카운트 초기화

# 결과 출력 (처음 10개)
print(triples_data[:10])
print(query_data[:5])


['0, 0, 2650', '1, 0, 2153', '2, 0, 1929', '3, 0, 1768', '4, 0, 1345', '5, 0, 3118', '6, 0, 36', '7, 0, 1361', '8, 0, 1748', '9, 0, 3135']
['에너지 균형이란 무엇인가요?', '건강한 사람이 에너지 균형을 유지해야 하는 이유는 무엇인가요?', '에너지 균형을 달성하기 위한 일반적인 기간은 얼마인가요?', '에너지 섭취와 에너지 소비를 조절하는 방법은 무엇인가요?', '영양가 있는 식품의 예시는 무엇인가요?']


In [5]:
# ColBERT 학습을 위하여 학습 데이터를 파일에 저장
collection_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/colbert/collection.tsv'
query_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/colbert/query.tsv'
triples_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/colbert/triples'

with open(collection_file, 'w') as f:
  for i,item in enumerate(sample_ko_collection):
    f.write(f'{i}\t{item}\n')

with open(query_file, 'w') as f:
  for i,item in enumerate(query_data):
    f.write(f'{i}\t{item}\n')

with open(triples_file, 'w') as f:
  for i,item in enumerate(triples_data):
    f.write(f'[{item}]\n')

In [6]:
!cat /data/ephemeral/home/upstage-ai-advanced-ir2/colbert/triples

[0, 0, 2650]
[1, 0, 2153]
[2, 0, 1929]
[3, 0, 1768]
[4, 0, 1345]
[5, 0, 3118]
[6, 0, 36]
[7, 0, 1361]
[8, 0, 1748]
[9, 0, 3135]
[10, 0, 3858]
[11, 0, 3919]
[12, 0, 2946]
[13, 1, 553]
[14, 1, 3750]
[15, 1, 359]
[16, 1, 1046]
[17, 1, 3292]
[18, 1, 1083]
[19, 1, 3760]
[20, 1, 2953]
[21, 1, 1528]
[22, 1, 2464]
[23, 1, 3070]
[24, 1, 4040]
[25, 1, 746]
[26, 2, 2161]
[27, 2, 2165]
[28, 2, 2354]
[29, 2, 2800]
[30, 2, 1529]
[31, 2, 788]
[32, 2, 2010]
[33, 2, 3908]
[34, 2, 3040]
[35, 2, 1634]
[36, 2, 1356]
[37, 2, 3767]
[38, 2, 637]
[39, 3, 1718]
[40, 3, 1287]
[41, 3, 985]
[42, 3, 3335]
[43, 3, 2924]
[44, 3, 135]
[45, 3, 510]
[46, 3, 1593]
[47, 3, 1028]
[48, 3, 2478]
[49, 3, 4216]
[50, 3, 1647]
[51, 3, 667]
[52, 4, 4088]
[53, 4, 1708]
[54, 4, 485]
[55, 4, 1023]
[56, 4, 2576]
[57, 4, 3893]
[58, 4, 2285]
[59, 4, 1705]
[60, 4, 874]
[61, 4, 2363]
[62, 4, 2743]
[63, 4, 1407]
[64, 4, 1811]
[65, 5, 859]
[66, 5, 1080]
[67, 5, 135]
[68, 5, 2067]
[69, 5, 3993]
[70, 5, 2484]
[71, 5, 3673]
[72, 5, 4018]
[73

In [7]:
!ls -l

total 9040
-rw-r--r-- 1 root root     115 Oct 14 12:13  OPENAI_API_KEY.env
-rw-r--r-- 1 root root    3057 Oct 14 11:32  README.md
-rw-r--r-- 1 root root 1922152 Oct 15 13:36 'Vector 유사도 검색 실습 - Elasticsearch.ipynb'
-rw-r--r-- 1 root root   27704 Oct 14 18:23  chapter_11.ipynb
drwxr-xr-x 2 root root    4096 Oct 15 14:57  colbert
-rw-r--r-- 1 root root  622051 Oct 15 15:46  colbert_ir.ipynb
-rw-r--r-- 1 root root 2619590 Oct 15 16:54 'colbert_reranking copy.ipynb'
-rw-r--r-- 1 root root 2619590 Oct 15 16:03  colbert_reranking.ipynb
-rw-r--r-- 1 root root 1143978 Oct 15 15:46  colbert_train.ipynb
-rw-r--r-- 1 root root       1 Oct 14 11:33  colbertv2_rag.ipynb
-rw-r--r-- 1 root root   10387 Oct 15 12:45  crossencoder_dataset.ipynb
-rw-r--r-- 1 root root    3753 Oct 15 14:17  crossencoder_train.ipynb
drwxr-xr-x 2 root root    4096 Oct 15 15:02  data
drwxr-xr-x 2 root root    4096 Oct 15 16:57  experiments
-rw-r--r-- 1 root root    9220 Oct 14 11:33  llama_batch50.ipynb
drwxr-xr-x 2 root ro

### 3.2 새로 만든 데이터로 모델 학습
90개의 소규모 데이터로 학습 진행

In [7]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Trainer

if __name__=='__main__':
    with Run().context(RunConfig(nranks=1, experiment="sample_ko_new_2")): # sentence-transformer-klue

        config = ColBERTConfig(
            bsize=24,
            root="experiments",
        )

        trainer = Trainer(
            triples=triples_file,
            queries=query_file,
            collection=collection_file,
            config=config,
        )

        # Pretrained model을 한국어 기반 모델로 설정해 준다.
        checkpoint_path = trainer.train(checkpoint='hunkim/sentence-transformer-klue') # hunkim/sentence-transformer-klue
        #checkpoint_path = trainer.train()

        print(f"Saved checkpoint to {checkpoint_path}...")


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 1,
    "kmeans_niters": 4,
    "resume": false,
    "pool_factor": 1,
    "clustering_mode": "hierarchical",
    "protected_tokens": 0,
    "similarity": "cosine",
    "bsize": 24,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 220,
    "mask_punctuation": true,
    "checkpoint

/data/ephemeral/home/miniconda3/envs/colbert/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of HF_ColBERT were not initialized from the model checkpoint at hunkim/sentence-transformer-klue and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/ephemeral/home/miniconda3/envs/colbert/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disab


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: 에너지 균형이란 무엇인가요?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([    0, 31500,  4291,  5238,  2052,  2241,  3890,  2179, 18119,    35,
            2,     4,     4,     4,     4,     4,     4,     4,     4,     4,
            4,     4,     4,     4,     4,     4,     4,     4,     4,     4,
            4,     4], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

#>>>    16.91 10.11 		|		 6.800000000000001
[Oct 16, 07:20:18] 0 0.006743285339325666
#>>>    16.15 8.67 		|		 7.479999999999999
[Oct 16, 07:20:18] 1 0.006740357880946249
#>>>    17.14 8.13 		|		 9.01
[Oct 16, 07:20:18] 2 0.006736460899547208
#>>>    17.86 7.63 		|		 10.23
[Oct 16, 07:20:18] 3 0.0067300727504966
#>>>    16.35 9.24 		|		 7.110000000000001
[Oct 16, 07:20:19] 4 0.006730190669819848
#>>>    19

In [8]:
# 학습된 모델의 위치
!find experiments/sample_ko_new -name colbert

experiments/sample_ko_new/none/2024-10/15/16.58.14/checkpoints/colbert
